# Spark Tutorial
Using: https://spark.apache.org/docs/latest/sql-data-sources-csv.html

## Aufgabe 1
Erstelle ein kleines Programm, dass die Codezeilen des aktuellen Files ausliest

In [1]:
# Wir wollen DataFrames nutzen und dafür brauchen wir eine SparkSession
# Import SparkSession
from pyspark.sql import SparkSession

# Create SparkSession
spark = SparkSession.builder\
    .master("local[1]")\
    .appName("Datenbanken mit Spark")\
    .getOrCreate() # getOrCreate liefert existierende Session, wenn es schon eine gibt

In [2]:
spark

In [13]:
# Inhalt des Directories anschauen
import os
for subdir, dirs, files in os.walk("./"):
    for file in files:
        print(file)
        
# !ls geht schneller 

SparkTutorial.ipynb
studibier.csv
PARSO-CACHE-LOCK
4305da1ea25c27fce08bd14001b76fd54fe42a0724bbd5168c76680a56eda5be-bc1435a60242c8b234f9abd81524755d819b9ea7a309f31f8888a73a7996f4e9.pkl
4305da1ea25c27fce08bd14001b76fd54fe42a0724bbd5168c76680a56eda5be-228436b9ebba4431fe2c0b886410cb0f7bec9d8800a7dd4c093a9a723ab46113.pkl
4305da1ea25c27fce08bd14001b76fd54fe42a0724bbd5168c76680a56eda5be-349f2ce27ea32e4072f92dbeec012d3898ee6ce80aff1962bdd10be515f67b14.pkl
4305da1ea25c27fce08bd14001b76fd54fe42a0724bbd5168c76680a56eda5be-162123e7cf80be5fa4925dc8d57ee63a0947eecb4d50441ca42fe4c5f252f0b0.pkl
4305da1ea25c27fce08bd14001b76fd54fe42a0724bbd5168c76680a56eda5be-c0e5518771ca845ce421dcd9749d2bd6c0f9b9e8303b0781d5b12f321067407c.pkl
4305da1ea25c27fce08bd14001b76fd54fe42a0724bbd5168c76680a56eda5be-1e6f73bfd15e0c88ba3a2f847d3e86b0e93bef06f5b930c57e2d485cdc9004cb.pkl
4305da1ea25c27fce08bd14001b76fd54fe42a0724bbd5168c76680a56eda5be-fc45f2959e780fafa5f42aa51b14115ec0f6b1236581ad99b0712a63e463d956.pkl
4305da1ea25

In [12]:
!pwd
!ls

/home/jovyan
SparkTutorial.ipynb  studibier.csv


In [14]:
# File einlesen
textFile = spark.read.text("SparkTutorial.ipynb")

In [16]:
# Zählen, wie viele Zeilen im Textfile enthalten sind
textFile.count()

163

In [17]:
textFile.first()
# Ausgabe der ersten Zeile vom Notebook, siehe nächster Command

Row(value='{')

In [18]:
textFile.show()
#textFile.show(6)
#textFile.head()

+--------------------+
|               value|
+--------------------+
|                   {|
|          "cells": [|
|                   {|
|   "cell_type": "...|
|   "execution_cou...|
|   "id": "50c16e5d",|
|     "metadata": {},|
|      "outputs": [],|
|         "source": [|
|    "# Wir wollen...|
|    "# Import Spa...|
|    "from pyspark...|
|               "\n",|
|    "# Create Spa...|
|    "spark = Spar...|
|    "    .master(...|
|    "    .appName...|
|    "    .getOrCr...|
|                   ]|
|                  },|
+--------------------+
only showing top 20 rows



## Aufgabe 2
Lies die .csv Datei *studibier.csv* aus Moodle in Spark ein (und berechne das Durchschnittsalter)

In [23]:
df_sbier = spark.read.csv("studibier.csv")

In [27]:
df_sbier.show()

+-----------+--------------+------+
|        _c0|           _c1|   _c2|
+-----------+--------------+------+
|       Name| Lieblingsbier| Alter|
|      James|        Weizen|    23|
|Money Penny|      Brut IPA|    24|
|       Lara|         NEIPA|    22|
|       Finn|       Chimais|    17|
|      Peter|         NEIPA|    16|
|       Jane|         NEIPA|    22|
+-----------+--------------+------+



In [30]:
# Header in Column verrutsch
df_sbier = spark.read.option("header", "true").csv("studibier.csv")
df_sbier

df_sbier.show()

+-----------+--------------+------+
|       Name| Lieblingsbier| Alter|
+-----------+--------------+------+
|      James|        Weizen|    23|
|Money Penny|      Brut IPA|    24|
|       Lara|         NEIPA|    22|
|       Finn|       Chimais|    17|
|      Peter|         NEIPA|    16|
|       Jane|         NEIPA|    22|
+-----------+--------------+------+



In [33]:
# Datentyp anzeigen
df_sbier.dtypes

if df_sbier.dtypes[1][1] == "string":
    print("It's a string")

It's a string


In [35]:
df_sbier.printSchema()

root
 |-- Name: string (nullable = true)
 |--  Lieblingsbier: string (nullable = true)
 |--  Alter: string (nullable = true)



In [36]:
df_sbier= spark.read.csv("studibier.csv", header=True, inferSchema=True)
df_sbier

# inferSchema = Welcher Datentyp wäre angemessen?

DataFrame[Name: string,  Lieblingsbier: string,  Alter: double]

In [38]:
df_sbier.show()

+-----------+--------------+------+
|       Name| Lieblingsbier| Alter|
+-----------+--------------+------+
|      James|        Weizen|  23.0|
|Money Penny|      Brut IPA|  24.0|
|       Lara|         NEIPA|  22.0|
|       Finn|       Chimais|  17.0|
|      Peter|         NEIPA|  16.0|
|       Jane|         NEIPA|  22.0|
+-----------+--------------+------+



## Columns, Rows

In [47]:
df_sbier.columns

['Name', ' Lieblingsbier', ' Alter']

In [49]:
df_sbier.select(" Lieblingsbier").distinct().show()

+--------------+
| Lieblingsbier|
+--------------+
|       Chimais|
|      Brut IPA|
|         NEIPA|
|        Weizen|
+--------------+



In [54]:
#df_sbier.agg({' Alter':'avg'}).show()
df_sbier.groupBy(" Lieblingsbier").agg({' Alter':'avg'}).show()

+--------------+-----------+
| Lieblingsbier|avg( Alter)|
+--------------+-----------+
|       Chimais|       17.0|
|      Brut IPA|       24.0|
|         NEIPA|       20.0|
|        Weizen|       23.0|
+--------------+-----------+



In [55]:
# Fixen von Quelldaten
df_sbier.withColumnRenamed(" Alter", "Alter").agg({"Alter": "avg"}).show()

+------------------+
|        avg(Alter)|
+------------------+
|20.666666666666668|
+------------------+



In [60]:
#df_sbier.select(" Lieblingsbier", " Alter").show()
df_sbier.select(" Lieblingsbier", " Alter").distinct().show()
# distinct() vergleicht jeder Zeile miteinander

+--------------+------+
| Lieblingsbier| Alter|
+--------------+------+
|         NEIPA|  22.0|
|         NEIPA|  16.0|
|        Weizen|  23.0|
|      Brut IPA|  24.0|
|       Chimais|  17.0|
+--------------+------+



In [62]:
from pyspark.sql.functions import lit
df_sbier.withColumn("Alter nach 2 Jahren", df_sbier[" Alter"]+2).withColumn("Eins", lit(1)).show()

+-----------+--------------+------+-------------------+----+
|       Name| Lieblingsbier| Alter|Alter nach 2 Jahren|Eins|
+-----------+--------------+------+-------------------+----+
|      James|        Weizen|  23.0|               25.0|   1|
|Money Penny|      Brut IPA|  24.0|               26.0|   1|
|       Lara|         NEIPA|  22.0|               24.0|   1|
|       Finn|       Chimais|  17.0|               19.0|   1|
|      Peter|         NEIPA|  16.0|               18.0|   1|
|       Jane|         NEIPA|  22.0|               24.0|   1|
+-----------+--------------+------+-------------------+----+



In [63]:
df_sbier.head(5)
# Die neuen Spalten davor existierten nur temporär In-Memory
# Der Datencode wurde nicht verändert, sondern nur die Ausgabe angepasst

[Row(Name='James',  Lieblingsbier=' Weizen',  Alter=23.0),
 Row(Name='Money Penny',  Lieblingsbier=' Brut IPA',  Alter=24.0),
 Row(Name='Lara',  Lieblingsbier=' NEIPA',  Alter=22.0),
 Row(Name='Finn',  Lieblingsbier=' Chimais',  Alter=17.0),
 Row(Name='Peter',  Lieblingsbier=' NEIPA',  Alter=16.0)]

In [64]:
df_sbierMitMuell = df_sbier.withColumn("Alter nach 2 Jahren", df_sbier[" Alter"]+2).withColumn("Eins", lit(1))
df_sbierMitMuell.show()

+-----------+--------------+------+-------------------+----+
|       Name| Lieblingsbier| Alter|Alter nach 2 Jahren|Eins|
+-----------+--------------+------+-------------------+----+
|      James|        Weizen|  23.0|               25.0|   1|
|Money Penny|      Brut IPA|  24.0|               26.0|   1|
|       Lara|         NEIPA|  22.0|               24.0|   1|
|       Finn|       Chimais|  17.0|               19.0|   1|
|      Peter|         NEIPA|  16.0|               18.0|   1|
|       Jane|         NEIPA|  22.0|               24.0|   1|
+-----------+--------------+------+-------------------+----+



In [65]:
# .drop() ist ebenfalls nur In-Memory
df_sbierMitMuell.drop("Alter nach 2 Jahren").show

<bound method DataFrame.show of DataFrame[Name: string,  Lieblingsbier: string,  Alter: double, Eins: int]>

In [66]:
df_sbierMitMuell.show()

+-----------+--------------+------+-------------------+----+
|       Name| Lieblingsbier| Alter|Alter nach 2 Jahren|Eins|
+-----------+--------------+------+-------------------+----+
|      James|        Weizen|  23.0|               25.0|   1|
|Money Penny|      Brut IPA|  24.0|               26.0|   1|
|       Lara|         NEIPA|  22.0|               24.0|   1|
|       Finn|       Chimais|  17.0|               19.0|   1|
|      Peter|         NEIPA|  16.0|               18.0|   1|
|       Jane|         NEIPA|  22.0|               24.0|   1|
+-----------+--------------+------+-------------------+----+



In [85]:
nullWerteDaten = [("Jenny", None, 22.0, 24.0, 1),("Jenny", None, None, 24.0, 1),("Jenny", None, None, None, 1)]
nwdSchema = df_sbierMitMuell.schema
df_bierWithNull = df_sbierMitMuell.union(spark.createDataFrame(data=nullWerteDaten, schema=nwdSchema))
df_bierWithNull.show()

+-----------+--------------+------+-------------------+----+
|       Name| Lieblingsbier| Alter|Alter nach 2 Jahren|Eins|
+-----------+--------------+------+-------------------+----+
|      James|        Weizen|  23.0|               25.0|   1|
|Money Penny|      Brut IPA|  24.0|               26.0|   1|
|       Lara|         NEIPA|  22.0|               24.0|   1|
|       Finn|       Chimais|  17.0|               19.0|   1|
|      Peter|         NEIPA|  16.0|               18.0|   1|
|       Jane|         NEIPA|  22.0|               24.0|   1|
|      Jenny|          null|  22.0|               24.0|   1|
|      Jenny|          null|  null|               24.0|   1|
|      Jenny|          null|  null|               null|   1|
+-----------+--------------+------+-------------------+----+



In [87]:
df_bierWithNull.show()
# Dropen aller Null-Zeilen
df_bierWithNull.na.drop(how="any").show()

+-----------+--------------+------+-------------------+----+
|       Name| Lieblingsbier| Alter|Alter nach 2 Jahren|Eins|
+-----------+--------------+------+-------------------+----+
|      James|        Weizen|  23.0|               25.0|   1|
|Money Penny|      Brut IPA|  24.0|               26.0|   1|
|       Lara|         NEIPA|  22.0|               24.0|   1|
|       Finn|       Chimais|  17.0|               19.0|   1|
|      Peter|         NEIPA|  16.0|               18.0|   1|
|       Jane|         NEIPA|  22.0|               24.0|   1|
|      Jenny|          null|  22.0|               24.0|   1|
|      Jenny|          null|  null|               24.0|   1|
|      Jenny|          null|  null|               null|   1|
+-----------+--------------+------+-------------------+----+

+-----------+--------------+------+-------------------+----+
|       Name| Lieblingsbier| Alter|Alter nach 2 Jahren|Eins|
+-----------+--------------+------+-------------------+----+
|      James|        We